In [1]:
# to make the export go to the fitbit app, click on your icon, and order a google takeout 
# took like 8 hours the last time

In [2]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")+1]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath)
from utils import exportsDataPath, workingDataPath, writeWorkingHRDfParquet

import pandas as pd

fitbitHRWorkingDataPath = workingDataPath + 'fitbit/hr/'

# Get the list of all files and directories
exportFilePath = exportsDataPath + "fitbit/17-9-24/takeout-20240917T195619Z-001/Takeout/Fitbit/Global Export Data/"
dir_list = os.listdir(exportFilePath)


/home/chowder/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
# read in the existing files and make the existing df
workingDataFiles = os.listdir(fitbitHRWorkingDataPath)
columnNames = ["sampleDT", "confidence", "value"]
dfSoFar = pd.DataFrame(columns=columnNames).set_index("sampleDT")

In [4]:
for dataFileName in workingDataFiles:
    dfSoFar = pd.concat([dfSoFar, pd.read_parquet(fitbitHRWorkingDataPath + dataFileName)]) 

dfSoFar = dfSoFar[~dfSoFar.index.duplicated(keep="first")].sort_index()

print(f"read in {len(dfSoFar)} rows from {len(workingDataFiles)} files")

read in 16052785 rows from 43 files


In [5]:
# get a list of unique dates in the index
#   removing the latest 3 days since they might be incomplete
if len(dfSoFar) > 0:
    datesSoFar = sorted(list(set(dfSoFar.index.date)))[:-3]
    print(datesSoFar[-1])
    hrFilenames = [x for x in dir_list if x.split("-")[0] == "heart_rate"]
    hrFilenames = [x for x in hrFilenames if pd.to_datetime(x[11:-5]).date() not in datesSoFar]
else:
    hrFilenames = [x for x in dir_list if x.split("-")[0] == "heart_rate"]

2024-09-14


In [6]:
#takes like 50 seconds to make 16 million rows
import json
import pytz
from datetime import datetime


def to_iso(s):
    return '20' + s[6:8] + '-' + s[0:2] + '-' + s[3:5] + s[8:]

samplesCount = 0
samplesList = []
for fn in hrFilenames:
    data = json.load(open(exportFilePath + fn))
    for row in data:
        sampleDT = datetime.fromisoformat(to_iso(row["dateTime"]))
        samplesList.append([sampleDT, 
                            row["value"]["confidence"], 
                            row["value"]["bpm"]])
        samplesCount += 1
        if samplesCount % 1_000_000 == 0:
            print(f"added {samplesCount} samples so far")
print(f"added {samplesCount} samples")

added 32915 samples


In [7]:
import pandas as pd

columnNames = ["sampleDT", "confidence", "value"]
samplesList = sorted(samplesList, key=lambda x: x[0]) #sort by timestamp

fitbitHRdf = pd.DataFrame(data=samplesList, columns=columnNames)
fitbitHRdf = fitbitHRdf.set_index("sampleDT")
fitbitHRdf.index = fitbitHRdf.index.tz_localize('UTC')
fitbitHRdf.index = pd.to_datetime(fitbitHRdf.index)

fitbitHRdf["confidence"] = fitbitHRdf["confidence"].astype('uint8')
fitbitHRdf["value"] = fitbitHRdf["value"].astype('uint8')
fitbitHRdf.dtypes

confidence    uint8
value         uint8
dtype: object

In [8]:
# concatenate the df's
# remove duplicate indexes
fitbitHRdf = pd.concat([dfSoFar, fitbitHRdf])
fitbitHRdf = fitbitHRdf[~fitbitHRdf.index.duplicated(keep="first")].sort_index()

In [9]:
writeWorkingHRDfParquet('fitbit', fitbitHRdf)

hashes match for 2020-05-13T100219-0700_2020-06-15T175149-0700_9b3f3630_.parquet.gzip
hashes match for 2020-06-15T175149-0700_2020-07-19T103931-0700_15ed86cc_.parquet.gzip
hashes match for 2020-07-19T103931-0700_2020-08-22T011641-0700_0c4966ee_.parquet.gzip
hashes match for 2020-08-22T011641-0700_2020-09-25T101037-0700_efba1909_.parquet.gzip
hashes match for 2020-09-25T101037-0700_2020-10-28T182707-0700_5f1b0c2b_.parquet.gzip
hashes match for 2020-10-28T182707-0700_2020-12-01T022613-0700_ad3308cd_.parquet.gzip
hashes match for 2020-12-01T022613-0700_2021-01-04T062220-0700_1bb757ba_.parquet.gzip
hashes match for 2021-01-04T062220-0700_2021-02-05T204607-0700_691ba674_.parquet.gzip
hashes match for 2021-02-05T204607-0700_2021-03-10T204558-0700_033c3397_.parquet.gzip
hashes match for 2021-03-10T204558-0700_2021-04-12T181209-0700_8473fbe8_.parquet.gzip
hashes match for 2021-04-12T181209-0700_2021-05-15T211855-0700_75a7df30_.parquet.gzip
hashes match for 2021-05-15T211855-0700_2021-06-17T102

In [10]:
import hashlib
import pickle
# Function to compute a short hash of a Python object
def short_hash(obj, length=8):
    # Serialize the object using pickle
    obj_bytes = pickle.dumps(obj)
    
    # Compute MD5 hash of the serialized object
    hash_obj = hashlib.md5(obj_bytes)
    
    # Return the hash truncated to the specified length
    return hash_obj.hexdigest()[:length]

In [11]:
print(short_hash(fitbitHRdf))
print(short_hash(fitbitHRdf) + " " + "test")
print(short_hash(fitbitHRdf) + " " + fitbitHRdf.iloc[0].name.strftime('%Y-%m-%dT%H%M%S%z'))
print(fitbitHRdf.iloc[0].name.strftime('%Y-%m-%dT%H%M%S%z') + " " + short_hash(fitbitHRdf))

7379ceed
7379ceed test
7379ceed 2020-05-13T100219-0700
2020-05-13T100219-0700 7379ceed


In [17]:
ilocdf = fitbitHRdf.iloc[0:100_000]
# print(ilocdf.iloc[0])
print(short_hash(ilocdf))
print(short_hash(ilocdf) + " " + "test")
print(short_hash(ilocdf) + " " + ilocdf.iloc[0].name.strftime('%Y-%m-%dT%H%M%S%z'))

#once iloc is called once 
print(ilocdf.iloc[0].name.strftime('%Y-%m-%dT%H%M%S%z') + " " + short_hash(ilocdf))
print(short_hash(ilocdf) + " " + ilocdf.iloc[0].name.strftime('%Y-%m-%dT%H%M%S%z'))
print(short_hash(ilocdf) + " " + "test")
print(ilocdf.iloc[0])

d5b98ce5
d5b98ce5 test
d5b98ce5 2020-05-13T100219-0700
2020-05-13T100219-0700 30d49dfb
30d49dfb 2020-05-13T100219-0700
30d49dfb test
confidence     0
value         70
Name: 2020-05-13 10:02:19-07:00, dtype: object


In [29]:
import pandas as pd
import hashlib
import pickle
# Function to compute a short hash of a Python object
def short_hash(obj, length=8):
    # Serialize the object using pickle
    obj_bytes = pickle.dumps(obj)
    
    # Compute MD5 hash of the serialized object
    hash_obj = hashlib.md5(obj_bytes)
    
    # Return the hash truncated to the specified length
    return hash_obj.hexdigest()[:length]

# Define the data with a timestamp column
data = {
    'Name': ['John', 'Alice', 'Bob', 'Eve', 'Charlie', 'David', 'Frank', 'Grace', 'Hannah', 'Ian'],
    'Age': [23, 25, 30, 22, 35, 29, 40, 27, 31, 28],
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'San Jose'],
    'Salary': [50000, 60000, 70000, 45000, 52000, 58000, 62000, 54000, 56000, 59000],
    'Department': ['HR', 'IT', 'Finance', 'HR', 'Marketing', 'IT', 'Finance', 'Marketing', 'HR', 'Finance'],
    'Timestamp': pd.date_range(start='2024-01-01', periods=10, freq='D')  # 10 sequential dates
}

# Create the DataFrame
testdf = pd.DataFrame(data)


slicedf = testdf.iloc[0:5]

print(short_hash(slicedf))
pi = slicedf
pic = slicedf.copy()
print(str(slicedf.iloc[0]))
print(short_hash(slicedf))
print(short_hash(pi))
print(short_hash(pic))


c2e90c5d
Name                         John
Age                            23
City                     New York
Salary                      50000
Department                     HR
Timestamp     2024-01-01 00:00:00
Name: 0, dtype: object
59b6270c
59b6270c
c2e90c5d


In [31]:
# Create the DataFrame
testdf = pd.DataFrame(data)


slicedf = testdf.iloc[0:5]

print(short_hash(slicedf.values))
pi = slicedf
pic = slicedf.copy()
print(str(slicedf.iloc[0]))
print(short_hash(slicedf.values))
print(short_hash(pi.values))
print(short_hash(pic.values))


f6189ca7
Name                         John
Age                            23
City                     New York
Salary                      50000
Department                     HR
Timestamp     2024-01-01 00:00:00
Name: 0, dtype: object
f6189ca7
f6189ca7
f6189ca7
